Indian General Elections 2019: Data Analysis And Prediction Using Machine Learning

# *Introduction:*

The 2019 Indian general election was held in seven phases from 11 April to 19 May 2019 to constitute the 17th Lok Sabha. The votes were counted, and the result was declared on 23 May. About 911 million people were eligible to vote, and voter turnout was over 67 percent.The highest ever and the highest ever participation by women voters. The Bharatiya Janata Party won 303 seats, further increasing its substantial majority, and the BJP-led National Democratic Alliance (NDA) won 353 seats. The BJP won 37.36% of votes, while the NDA's combined ballot was 45% of the 60.37 crore votes that were polled. The Indian National Congress won just 52 seats, and the Congress-led United Progressive Alliance won 91. Other parties and their alliances won 98 seats.
In this notebook, I have used data visualization tools such as Matplotlib, Plotly, and Seaborn to visualize how India's people voted during the 2019 elections. I have also experimented with Random Forest Classifier, Support Vector Machine, and K-Nearest Neighbor machine learning models to make predictions. The models' hyperparameters were first tuned manually, and then GridSearchCV was used to find the most optimum results. I have also evaluated the performance metrics of these models and compared them with each other.

# *Environment Setup*

In [5]:
# Import numpy, pandas for data manipulation
import numpy as np
import pandas as pd
from collections import Counter
import time

# Import plotly, matplotlib and seaborn as visualization tools
import matplotlib.pyplot as plt
from plotly import tools
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode
import seaborn as sns

# Import for resampling the data
from imblearn.over_sampling import SMOTE

# Import for scaling the data
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder

# Import machine learning models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier

# Import model evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Plot should appear inside the jupyter notebook
%matplotlib inline

init_notebook_mode(connected=True)


ModuleNotFoundError: No module named 'xgboost'

# Gathering The Data

The data for this project has been collected from Kaggle.

In [ ]:
# Loading the dataset
df = pd.read_csv("/content/LS_2.0.csv")
df.head()

In [ ]:
df.replace({'Not Available': np.nan}, inplace=True)

# Features
STATE,
CONSTITUENCY,
NAME,
WINNER,
PARTY,
SYMBOL,
GENDER,
CRIMINAL CASES,
AGE,
CATEGORY,
EDUCATION,
ASSETS,
LIABILITIES,
GENERAL VOTES,
POSTAL VOTES,
TOTAL VOTES,
OVER TOTAL ELECTORS IN CONSTITUENCY,
OVER TOTAL VOTES POLLED IN CONSTITUENCY,
TOTAL ELECTORS.

# Data Preprocessing and Exploratory Data Analysis (EDA)

In [ ]:
df.info()

In [ ]:
df.columns = df.columns.str.replace('\r','')
df.columns = df.columns.str.replace('\n','')

In [ ]:
df.describe()

In [ ]:
# Checking to see if the dataset contains any null values. We need to exclude NOTA votes while checking it.
df = df[df['PARTY']!= 'NOTA']
df = df.dropna()
df.isna().sum()

In [ ]:
df.head(10)

In [ ]:
df.dtypes

In [ ]:
def value_cleaner(x):
    try:
        str_temp = (x.split('Rs')[1].split('\n')[0].strip())
        str_temp_2 = ''
        for i in str_temp.split(","):
            str_temp_2 = str_temp_2+i
        return str_temp_2
    except:
        x = 0
        return x
df['ASSETS'] = df['ASSETS'].apply((value_cleaner))
df['LIABILITIES'] = df['LIABILITIES'].apply((value_cleaner))

In [ ]:
df['EDUCATION'].value_counts()

In [ ]:
# Removing the \n from 'Post Graduate\n'
df['EDUCATION'].replace(to_replace='Post Graduate\r\n', value='Post Graduate', inplace=True)

# 'Graduate Professional' are Graduates, so replacing 'Graduate Professional' with 'Graduate'
df['EDUCATION'].replace(to_replace='Graduate Professional', value='Graduate', inplace=True)

# 'Literate' = 8th Pass in our society
df['EDUCATION'].replace(to_replace='Literate', value='8th Pass', inplace=True)

# Any education level below 8th pass is illiterate
df['EDUCATION'].replace(to_replace='5th Pass', value='Illiterate', inplace=True)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['CRIMINALCASES'] = df['CRIMINALCASES'].astype(int, errors='raise')
df['ASSETS'] = df['ASSETS'].astype(float, errors='raise')
df['LIABILITIES'] = df['LIABILITIES'].astype(float, errors='raise')

In [ ]:
df.head()

# Data Analysis
State and Constituency by total electors

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Ensure plotly's renderer is set to 'colab' for Google Colab
pio.renderers.default = 'colab'

# Load the CSV file
df = pd.read_csv('/content/LS_2.0.csv')

# Create the state and constituency dataframe
state_df = df[['STATE', 'CONSTITUENCY', 'TOTAL ELECTORS']]

# Create the sunburst chart
state_fig = px.sunburst(state_df,
                        path=['STATE', 'CONSTITUENCY'],
                        values='TOTAL ELECTORS',
                        color='TOTAL ELECTORS',
                        color_continuous_scale=px.colors.sequential.Sunset)  # Use a diverse and vibrant color scale

# Update layout with enhanced visual aesthetics
state_fig.update_layout(
    title_text='State and Constituency by Total Electors',
    title_font=dict(size=24, family='Courier New', color='#351f39'),
    font=dict(family='Courier New', color='#351f39'),
    paper_bgcolor='#f4f4f4',  # Set a light background for contrast
    plot_bgcolor='#ffffff',
    margin=dict(t=50, l=25, r=25, b=25)  # Adjust margins for a clean look
)

# Show the plot using plotly's show function
pio.show(state_fig)



# Lok Sabha 2019 Election Results : Top 20 Political Parties

In [ ]:
# Number of seats won by each party
result = df[df['WINNER'] == 1].groupby('PARTY')['WINNER'].size()
result_df = pd.DataFrame(data=result).sort_values(by="WINNER", ascending=False)
result_df.reset_index(level=0, inplace=True)

# Visualize the party-wise election results
result_fig = px.bar(result_df,
                    x=result_df['PARTY'][:20],
                    y=result_df["WINNER"][:20],
                    color=result_df['WINNER'][:20],
                    height=500,
                    color_continuous_scale='Oryel')
result_fig.update_layout(title_text='Top 20 Party-wise election results',
                         font_family="Courier New",
                         font_color="#351f39",
                         title_font_family="Courier New",
                         title_font_color="#351f39",
                         xaxis_title="Party",
                         yaxis_title="Seats")
result_fig.show()


Bharatiya Janata Party (BJP) emerged as the single largest party after winning 300 seats. BJP is also the single largest party in terms of total vote share in Lok Sabha Election 2019. Congress secured the second position after winning 52 seats. After the landslide victory of Bharatiya Janata Party in the 17th Lok Sabha election; Narendra Modi is the 15th Prime Minister of India. BJP got the thumping majority in the 17th Lok Sabha election. Worth to mention that maximum seats in Lok Sabha can be 552; comprises 530 from states, 20 from Union Territories and 2 nominated members. Currently, only 543 members are elected for Lok Sabha from different states and UTs for the 17th Lok Sabha.

# Party-Wise Vote Share

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Create a dataframe with the total number of votes won by each party
result = df[df['WINNER'] == 1].groupby('PARTY')['WINNER'].size()
vote_Share = df.groupby('PARTY')['TOTAL\nVOTES'].sum()
vote_Share_df = pd.DataFrame(data=vote_Share, index=vote_Share.index).sort_values(by="TOTAL\nVOTES", ascending=False)
vote_Share_df.reset_index(level=0, inplace=True)

# Sort out the top five parties according to the total number of votes
vote_share_top5 = df.groupby('PARTY')['TOTAL\nVOTES'].sum().nlargest(5).index.tolist()

# Create a method to label parties as 'Other' if they are not top five in total number of votes.
def sort_party(data):
    if data['PARTY'] not in vote_share_top5:
        return 'Other'
    else:
        return data['PARTY']

# Calling the method
df['Party New'] = df.apply(sort_party, axis=1)

# Visualize Party-wise vote share
vote_count = df.groupby('Party New')['TOTAL\nVOTES'].sum()
vote_count_fig = go.Figure(go.Pie(labels=vote_count.index,
                                  values=vote_count.values,
                                  pull=[0.1 if label == 'Other' else 0 for label in vote_count.index],
                                  marker=dict(colors=px.colors.qualitative.Bold,  # Use a bold qualitative color palette
                                              line=dict(color="#000000",
                                                        width=2))))
vote_count_fig.update_layout(title_text='Party-wise Vote Share',
                             font_family="Courier New",
                             font_color="#351f39",
                             title_font_family="Courier New",
                             title_font_color="#351f39",
                             legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5))
vote_count_fig.show()



In [ ]:
df['Party New'].value_counts()



The Bharatiya Janata Party garnered 38.8% of the votes in the 2019 Lok Sabha elections. This is the highest vote share received by the party nation-wide in any Lok Sabha election since the party was (re)formed in 1980. In contrast, the Congress party failed to improve on its vote share from 2014 and gathered 20.2% of the total votes. The dominant victory of the BJP and its allies — by repeating the 2014 win, and then some — suggests that the era of coalitions (and/or a central government with a relatively weak national party in power) determining the balance of power at Delhi has decisively come to an end

### Age Distribution Of Candidates

In [ ]:
# Visualize the age distribution of candidates
age_distribution_fig = px.histogram(df,
                                    x="AGE",
                                    color_discrete_sequence=['indianred'])
age_distribution_fig.update_layout(title_text='Age Distribution of Candidates',
                                   xaxis_title="Distribution",
                                   yaxis_title="Age")
age_distribution_fig.update_layout(font_family="Courier New",
                                   font_color="#351f39",
                                   title_font_family="Courier New",
                                   title_font_color="#351f39")
age_distribution_fig.show()

# Caste Distribution Of Candidates

In [ ]:
# Create a dataframe containing the caste distribution of winners
winner_caste = df[df['WINNER'] == 1].groupby('CATEGORY')['WINNER'].size()
winner_caste_df = pd.DataFrame(data=winner_caste)
winner_caste_df.reset_index(level=0, inplace=True)
winner_caste_df['Category'] = 'Winner'

# Create a dataframe containing the caste distribution of total number of candidates
total_caste = df.groupby('CATEGORY').size()
total_caste_df = pd.DataFrame(data=total_caste)
total_caste_df.reset_index(level=0, inplace=True)
total_caste_df['Category'] = 'Total'
total_caste_df.rename(columns={0: "WINNER"}, inplace=True)

# Concatenate the two dataframes
winner_caste_df = pd.concat((winner_caste_df, total_caste_df))

# Visualize the results
caste_fig = px.bar(winner_caste_df,
                   x='CATEGORY',
                   y='WINNER',
                   color='Category',
                   barmode='group')
caste_fig.update_layout(font_family="Courier New",
                        font_color="#351f39",
                        title_text='Caste distribution of candidates',
                        title_font_family="Courier New",
                        title_font_color="#351f39",
                        xaxis_title='Caste',
                        yaxis_title='Count')
caste_fig.show()


# Top ten party-wise criminal records

In [ ]:
# Fill NaNs with 0 or another value
df['CRIMINAL\nCASES'] = pd.to_numeric(df['CRIMINAL\nCASES'], errors='coerce')
df.dropna(subset=['CRIMINAL\nCASES'], inplace=True)

In [ ]:
import pandas as pd
import plotly.express as px

# Ensure that your DataFrame is loaded
# df = pd.read_csv('/path/to/your/LS_2.0.csv')  # Uncomment if you need to load the data

# Convert 'CRIMINAL\nCASES' column to numeric
df['CRIMINAL\nCASES'] = pd.to_numeric(df['CRIMINAL\nCASES'], errors='coerce')

# Drop rows with NaN values in 'CRIMINAL\nCASES' column
df.dropna(subset=['CRIMINAL\nCASES'], inplace=True)

# Create a dataframe of criminal records
criminal_df = pd.DataFrame(data=df[df['WINNER'] == 1].groupby('PARTY')['CRIMINAL\nCASES'].sum())
criminal_df.sort_values(by='CRIMINAL\nCASES', ascending=False, inplace=True)
criminal_df.reset_index(level=0, inplace=True)

# Visualize the criminal records for the top 10 parties
criminal_fig = px.scatter(criminal_df[:10],
                          x='PARTY',
                          y='CRIMINAL\nCASES',
                          size='CRIMINAL\nCASES',
                          color='PARTY',
                          hover_data=['CRIMINAL\nCASES'])

# Update layout with enhanced visual aesthetics
criminal_fig.update_layout(title_text='Top 10 Party-wise Criminal Records',
                           font_family="Courier New",
                           font_color="#351f39",
                           title_font_family="Courier New",
                           title_font_color="#351f39",
                           xaxis_title="Party",
                           yaxis_title="Criminal Records")

# Display the figure
criminal_fig.show()


# Key battlegrounds of the election

Amethi (Uttar Pradesh): Known to be a traditional Congress bastion, BJP had fielded Union Minister Smriti Irani against Congress president Rahul Gandhi. The voting day in Amethi in the fifth phase was marked by charges of 'booth-rigging' by Irani and Congress president Rahul Gandhi being absent from the constituency. In 2014, Irani lost to Rahul Gandhi from the same seat with a vote-margin of 1.07 lakh votes or a vote-ratio of about 3:4. This time, her challenge has forced Rahul Gandhi to also seek election from Wayanad.

Varanasi (Uttar Pradesh): Given the significance of the seat, mid-polls rumours had emerged that the Congress' General Secretary for Uttar Pradesh East Priyanka Gandhi Vadra could contest the seat. However, the speculations were put to rest as Congress fielded Ajay Rai, hours before PM Modi addressed a massive road-show ahead of his nomination-filing. In the 2014 Parliamentary elections, PM Modi swept the seat by receiving 56.40% of votes and defeated Arvind Kejriwal by a by 3.37 lakh vote margin.

In [ ]:
# Battleground Amethi
amethi_df = df[df['CONSTITUENCY'] == 'AMETHI'][['NAME','TOTAL\nVOTES']]

# Battleground Varanasi
varanasi_df = df[df['CONSTITUENCY'] == 'VARANASI'][['NAME','TOTAL\nVOTES']]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,7))
amethi_df.plot(kind='bar', ax=axes[0], alpha=0.75, rot=0, legend=False, title='Battleground Amethi: Smriti Irani vs Rahul Gandhi', x='NAME')
varanasi_df.plot(kind='bar', ax=axes[1], alpha=0.75, rot=0, legend=False, title="Battleground Varanasi: Narendra Modi's Constituency", x='NAME')

1)This time things were different in Amethi. Smriti Irani defeated Rahul Gandhi by a substantial margin.

2)In Varanasi however the picture was similar with Narendra Modi winning the seat by a landslide.

# West Bengal:

West Bengal remained the newsmaker of the 2019 Lok Sabha polls, with violence including crude bombs, lathi charges and repeated political clashes and claims of attacks from candidates became the talking point in the state as polling was held in all seven phases spread across two months in the politically important state. Bengal has 42 Lok Sabha constituencies, of which the Trinamool won 34 in 2014. The Congress managed four and the Left two. The BJP secured only two, despite the Narendra Modi wave in the country that year. The state which was considered to be a bastion for the Left Wing has now become a stronghold for the Trinamool Congress.

In [ ]:
# Battleground West Bengal
bengal_df = pd.DataFrame(data=df[df['STATE'] == 'West Bengal'])
bengal_df_vote = pd.DataFrame(data=bengal_df.groupby('PARTY')['TOTAL\nVOTES'].sum())
bengal_df_vote.sort_values(by='TOTAL\nVOTES', ascending=False, inplace=True)
bengal_df_vote.reset_index(level=0, inplace=True)
bengal_df_vote['Category'] = bengal_df_vote.apply(sort_party, axis=1)
bengal_vote_count = bengal_df_vote.groupby('Category')['TOTAL\nVOTES'].sum()

vote_share_top5 = bengal_df_vote.groupby('PARTY')['TOTAL\nVOTES'].sum().nlargest(5).index.tolist()
# Visualize Party-wise vote share
bengal_fig = go.Figure(go.Pie(labels=bengal_vote_count.index,
                                  values=bengal_vote_count.values,
                                  pull=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
                                  marker=dict(line=dict(color="#000000",
                                                        width=2))))
bengal_fig.update_layout(title_text='West Bengal : Party-wise Vote Share',
                             font_family="Courier New",
                             font_color="#351f39",
                             title_font_family="Courier New",
                             title_font_color="#351f39")
bengal_fig.show()

The Trinamool Congress recorded its best-ever performance in 2014 Lok Sabha election in West Bengal as it swept 34 of the 42 seats decimating the Left in its victory march, while the BJP made impressive inroads winning two seats. The BJP left no stone unturned in attempting to make inroads into Mamata Banerjee's bastion while the Trinamool responded by not ceding an inch. This time around BJP swept the 2019 Lok Sabha polls winning 18 seats out of 42 with a vote share of 41.6%. However it failed to dent the consolidated vote share of Mamata Banerjee's TMC

# How people voted in Bengal?

In [ ]:
# Create a dataframe containing the caste distribution of winners
winner_caste_bengal = bengal_df[bengal_df['WINNER'] == 1].groupby('CATEGORY')['WINNER'].size()
winner_caste_bengal_df= pd.DataFrame(data=winner_caste_bengal)
winner_caste_bengal_df.reset_index(level=0, inplace=True)

def education(data):
    if data['EDUCATION'] in ['5th Pass', 'Others', 'Illiterate', 'Not Available']:
        return 'Uneducated'
    elif data['EDUCATION'] in ['Graduate', 'Doctorate', 'Post Graduate']:
        return 'Highly Educated'
    else:
        return 'Educated'

bengal_df['Education Level'] = bengal_df.apply(education, axis=1)

winner_edu_bengal = bengal_df[bengal_df['WINNER'] == 1].groupby('Education Level')['WINNER'].size()
winner_edu_bengal_df= pd.DataFrame(data=winner_edu_bengal)
winner_edu_bengal_df.reset_index(level=0, inplace=True)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,7))
winner_caste_bengal_df.plot(kind='bar', ax=axes[0], alpha=0.75, rot=0, legend=False, title='West Bengal: Caste of winning candidates', x='CATEGORY')
winner_edu_bengal_df.plot(kind='bar', ax=axes[1], alpha=0.75, rot=0, legend=False, title='West Bengal: Education of winning candidates', x='Education Level')

1)Most of the winning candidates are from General caste.

2)When it comes to education most of the winning candidates are highly educated. The level of education of an candidate is a key factor in West Bengal

Begusarai (Bihar): The Begusarai constituency, which is among the most talked about seats in Bihar, recorded a comparatively high turnout of 61.27%, as per the EC. The constituency has been under a microscope as it is witnessing the electoral debut of CPI candidate Kanhaiya Kumar, the controversial former JNUSU president who is pitted against BJP leader and union minister Giriraj Singh. The Mahagathbandhan's nominee is Tanweer Hasan (RJD)

Bangalore South (Karnataka): The saffron bastion will see BJP's Tejasvi Surya take on Congress’ BK Hariprasad. After the untimely death of BJP leader and union minister Ananth Kumar who had held the constituency since 1996, BJP has fielded a young face in Tejasvi Surya. The late Ananth Kumar had beaten Infosys co-founder Nandan Nilekani, then a Congress candidate, in 2014.

Wayanad (Kerala): Congress chief Rahul Gandhi contesting from this constituency apart from Congress' bastion in Amethi has made the relatively new seat becoming a national talking point. Against him are PP Suneer of CPI(M) and Thushar Vellappally of Bharath Dharma Jana Sena (an ally of BJP). Interestingly, Congress chief will be fighting with three namesakes: two Independent candidates, K.E. Rahul Gandhi and K.M. Shivprasad Gandhi, and K. Raghul Gandhi of the Agila India Makkal Kazhagam party.

Bhopal (Madhya Pradesh): A BJP bastion, this constituency has taken centrestage ever since the BJP fielded Malegaon blast-accused Sadhvi Pragya from the seat against Congress veteran Digvijaya Singh. Singh has been CM of Madhya Pradesh and is contesting an election after a gap of almost 19 years, the last being the assembly election he contested in 2003. He was nominated to the Rajya Sabha in 2014 from Madhya Pradesh. The BJP however has consistently been winning the seat of Bhopal since 1989. BJP's Alok Sanjar is the sitting MP from the Bhopal Lok Sabha seat. What makes the contest especially interesting, however, is the fact that Digvijaya Singh was one of the most significant proposers of 'Saffron terror/Hindu terror' which cited the Malegaon blasts among others

In [ ]:
begusarai_df = df[df['CONSTITUENCY'] == 'BEGUSARAI'][['NAME','TOTAL\nVOTES']]
bangalore_df = df[df['CONSTITUENCY'] == 'BANGALORE SOUTH'][['NAME','TOTAL\nVOTES']]
wayanad_df = df[df['CONSTITUENCY'] == 'WAYANAD'][['NAME','TOTAL\nVOTES']]
bhopal_df = df[df['CONSTITUENCY'] == 'BHOPAL'][['NAME','TOTAL\nVOTES']]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15,15))
begusarai_df.plot(kind='bar', ax=axes[0,0], alpha=0.75, rot=0, legend=False, title='Battleground Begusarai', x='NAME')
bangalore_df.plot(kind='bar', ax=axes[0,1], alpha=0.75, rot=0, legend=False, title="Battleground Bangalore South", x='NAME')
wayanad_df.plot(kind='bar', ax=axes[1,0], alpha=0.75, rot=0, legend=False, title='Battleground Wayanad', x='NAME')
bhopal_df.plot(kind='bar', ax=axes[1,1], alpha=0.75, rot=0, legend=False, title="Battleground Bhopal", x='NAME')

# Preparing the data

# Scaling the data

In [ ]:
# This is the dataset which will be used for fitting Machine Learning models
df.head()

In [ ]:
# Labeling parties with less than 10 candidates as others
less_candidates = []
for i, j in df['PARTY'].value_counts().items():
    if j <= 10:
        less_candidates.append(i)
len(less_candidates)

def small_party(data):
    if data in less_candidates:
        return 'Other'
    return data
df['PARTY'] = df['PARTY'].apply(small_party)
df['PARTY'].value_counts()

In [ ]:
X = df.drop(['NAME', 'SYMBOL', 'WINNER', 'GENERAL\nVOTES', 'POSTAL\nVOTES', 'TOTAL\nVOTES', 'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY',
             'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY', 'Party New', 'CONSTITUENCY'], axis=1)
y = df['WINNER']
X.head()

In [ ]:
# Creating training and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
categorical = []
numerical = []

# Converting Strings to Category
for label, content in X.items():
    if pd.api.types.is_string_dtype(content):
        X[label] = content.astype("category")
        categorical.append(label)

    if pd.api.types.is_numeric_dtype(content):
        numerical.append(label)

In [ ]:
# Initializing the StandardScaler()
scaler = MinMaxScaler()
encoder = OrdinalEncoder()

# Encoding the categorical columns
for cat in categorical:
    label_fit = encoder.fit(np.array(X[cat]).reshape(-1, 1))
    X_train[cat] = label_fit.transform(np.array(X_train[cat]).reshape(-1, 1))
    X_test[cat] = label_fit.transform(np.array(X_test[cat]).reshape(-1, 1))

# Scaling the dataset
scaler_fit = scaler.fit(X[numerical])
X_train[numerical] = scaler_fit.transform(X_train[numerical])
X_test[numerical] = scaler_fit.transform(X_test[numerical])
# Encoding the target
target_enc = LabelEncoder().fit(y_train)
y_train = target_enc.transform(y_train)
y_test = target_enc.transform(y_test)

# Upsampling the dataset

In [ ]:
Counter(y_train)

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)
counter = Counter(y_train)
print(counter)

# Machine learning model experimentation

In [ ]:
np.random.seed(42)

# Create a dictionary of the models to experiment
models = {
          "Random Forest" : RandomForestClassifier(n_jobs=-1),
          "Support Vector Machines" : SVC(),
          "K-Nearest Neighbors" : KNeighborsClassifier(),
          "Logistic Regression"  : LogisticRegression(),
          "XG Boosting" : xgb.XGBClassifier(n_jobs=-1),
          "Cat Boosting": CatBoostClassifier(),
          "Gradient Boosting"  : GradientBoostingClassifier(),
          "ADA Boosting" : AdaBoostClassifier(),
          "LG Boosting" : lgb.LGBMClassifier(),
          "Decision Trees" : DecisionTreeClassifier()
         }

# Create a function to fit and score the models
def fit_and_score(models, X_train, y_train, X_test, y_test):
    '''
    Fits and evaluates the machine learning models.
    models : a dict of different Scikit-Learn machine learning models.
    X_train : training data (no labels)
    y_train : training lables
    X_test : testing data (no labels)
    y_test : testing lables
    '''
    train_model_scores_acc = {}
    train_model_scores_pre = {}
    train_model_scores_rec = {}
    train_model_scores_f1 = {}

    test_model_scores_acc = {}
    test_model_scores_pre = {}
    test_model_scores_rec = {}
    test_model_scores_f1 = {}

    train_model_timing = {}

    for name, model in models.items():
        start = time.time()
        train_model_scores_acc[name] = np.mean(cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy')) * 100
        train_model_scores_pre[name] = np.mean(cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1, scoring='precision')) * 100
        train_model_scores_rec[name] = np.mean(cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1, scoring='recall')) * 100
        train_model_scores_f1[name] = np.mean(cross_val_score(model, X_train, y_train, cv=10, n_jobs=-1, scoring='f1')) * 100
        stop = time.time()
        timing = stop - start
        train_model_timing[name] = timing

        model.fit(X_train, y_train)
        pred = model.predict(X_test)

        test_model_scores_acc[name] = accuracy_score(y_test, pred) * 100
        test_model_scores_pre[name] = precision_score(y_test, pred) * 100
        test_model_scores_rec[name] = recall_score(y_test, pred) * 100
        test_model_scores_f1[name] = f1_score(y_test, pred) * 100

    # Create dictionary for oil scores
    scores = {'Training Accuracy': train_model_scores_acc,
              'Training Precision': train_model_scores_pre,
              'Training Recall': train_model_scores_rec,
              'Training F1': train_model_scores_f1,
              'Test Accuracy':  test_model_scores_acc,
              'Test Precision': test_model_scores_pre,
              'Test Recall': test_model_scores_rec,
              'Test F1': test_model_scores_f1,
              '10-Fold CV Timing (seconds)': train_model_timing
             }

    # Create dataframe of scores
    scores_df = pd.DataFrame(scores)

    scores_df.rename_axis('Model', inplace=True)
    scores_df.reset_index(inplace=True)
    scores_df.sort_values('Test Accuracy', ascending=False, inplace=True)
    return scores_df


In [ ]:
!pip install catboost


In [ ]:
scores_df=fit_and_score(models, X_train, y_train, X_test, y_test)

In [ ]:
scores_df


# Evaluation

In [ ]:
# Fitting the final model
final_model = xgb.XGBClassifier(n_jobs=-1)
final_model.fit(X_train, y_train)

# Making predictions
pred = final_model.predict(X_test)

In [ ]:
xgb.plot_importance(final_model)

# Classification report

In [ ]:
print(classification_report(y_test, pred))

# Confusion matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, pred), cmap= "Blues", annot=True, fmt='')

# ROC curve

In [ ]:
!pip install --upgrade scikit-learn


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt



In [ ]:
# Get the predicted probabilities for the positive class
y_prob = final_model.predict_proba(X_test)[:, 1]

# Compute ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()
